In [ ]:
! pip install liac-arff

In [ ]:
## Checks to be executed for each dataset
## 1. The unique values across the entire dataset should be 0 or 1 only
## 2. There should NOT be any columns in the co-variate space with all value equal. Let's remove those columns

In [3]:
import arff
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np

In [4]:
path = "/domino/datasets/local/CustomerSegmentation/public_datasets/enron/"

In [5]:
# Load the .arff file - With full data in one file
with open(path + 'enron.arff', 'r') as f:
    dataset = arff.load(f)
    
# Extract data and attributes
data = dataset['data']
attributes = dataset['attributes']

# Create DataFrame
columns = [attr[0] for attr in attributes]
df = pd.DataFrame(data, columns=columns)

In [6]:
print("Shape of the delicious.arff file :", df.shape)

Shape of the delicious.arff file : (1702, 1054)


In [12]:
### Get the position of the first Label columns
col_position = df.columns.get_loc("A.A8")  ## The first label column in the dataset
print(col_position)

1001


In [13]:
df.columns

Index(['0', '00', '000', '01', '02', '03', '04', '05', '06', '07',
       ...
       'C.C3', 'D.D10', 'D.D18', 'B.B13', 'D.D17', 'B.B10', 'C.C1', 'D.D4',
       'C.C13', 'D.D14'],
      dtype='object', length=1054)

In [16]:
df.iloc[:,:col_position].columns

Index(['0', '00', '000', '01', '02', '03', '04', '05', '06', '07',
       ...
       'workers', 'working', 'world', 'writer', 'writers', 'www', 'year',
       'years', 'yesterday', 'york'],
      dtype='object', length=1001)

In [17]:
##Check if the unique values across the entire co-variate space is 0 and 1 only or not
unique_values = np.unique(df.iloc[:, :col_position].values)
print(unique_values)

['0' '1']


In [18]:
## Check if the unique values in the mulit labels is 0 and 1 only or not
unique_values = np.unique(df.iloc[:,col_position:].values)
print(unique_values)

['0' '1']


In [19]:
## Check to drop columns from the dataset with same values for all rows

# Subset the first 1836 columns
features = df.iloc[:, :col_position]

# Identify columns with more than one unique value
non_constant_cols = features.loc[:, features.nunique(dropna=False) > 1]

print("Shape of the non_constant_cols :", non_constant_cols.shape)

# Concatenate with the remaining part of the DataFrame (e.g., label columns)
df_cleaned = pd.concat([non_constant_cols, df.iloc[:, col_position:]], axis=1)

print("Shape of df_cleaned :", df_cleaned.shape)

Shape of the non_constant_cols : (1702, 1001)
Shape of df_cleaned : (1702, 1054)


In [20]:
## No columns were deleted during constant value checks

In [21]:
df.iloc[:,:col_position].columns

Index(['0', '00', '000', '01', '02', '03', '04', '05', '06', '07',
       ...
       'workers', 'working', 'world', 'writer', 'writers', 'www', 'year',
       'years', 'yesterday', 'york'],
      dtype='object', length=1001)

In [22]:
##  Rename the co-variate feature space of the dataset

# Generate new column names
new_feature_names = [f"f_{i}" for i in range(col_position)]

# Assign the new names to the first 1836 columns
df_cleaned.columns.values[:col_position] = new_feature_names

In [23]:
df_cleaned.head(3)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,C.C3,D.D10,D.D18,B.B13,D.D17,B.B10,C.C1,D.D4,C.C13,D.D14
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
## Dataset is highly imbalanced

# Slice only the label columns
label_data = df_cleaned.iloc[:, col_position:]

# Calculate proportions of 0's and 1's for each column
for col in label_data.columns:
    value_counts = label_data[col].value_counts(normalize=True).sort_index()
    prop_0 = value_counts.get(0, 0)
    prop_1 = value_counts.get(1, 0)
    print(f"{col}: 0's = {prop_0:.4f}, 1's = {prop_1:.4f}")

A.A8: 0's = 0.9847, 1's = 0.0153
C.C9: 0's = 0.9624, 1's = 0.0376
B.B12: 0's = 0.9965, 1's = 0.0035
C.C11: 0's = 0.9847, 1's = 0.0153
C.C5: 0's = 0.9365, 1's = 0.0635
C.C7: 0's = 0.9512, 1's = 0.0488
B.B2: 0's = 0.4636, 1's = 0.5364
B.B3: 0's = 0.9753, 1's = 0.0247
D.D16: 0's = 0.9941, 1's = 0.0059
A.A7: 0's = 0.9853, 1's = 0.0147
D.D1: 0's = 0.9929, 1's = 0.0071
A.A4: 0's = 0.6868, 1's = 0.3132
C.C2: 0's = 0.9266, 1's = 0.0734
A.A3: 0's = 0.9031, 1's = 0.0969
A.A1: 0's = 0.4976, 1's = 0.5024
D.D9: 0's = 0.9894, 1's = 0.0106
D.D19: 0's = 0.9947, 1's = 0.0053
B.B8: 0's = 0.9871, 1's = 0.0129
D.D12: 0's = 0.9777, 1's = 0.0223
D.D6: 0's = 0.9877, 1's = 0.0123
C.C8: 0's = 0.9371, 1's = 0.0629
A.A6: 0's = 0.8966, 1's = 0.1034
B.B9: 0's = 0.9677, 1's = 0.0323
A.A5: 0's = 0.9436, 1's = 0.0564
C.C10: 0's = 0.9548, 1's = 0.0452
B.B1: 0's = 0.6005, 1's = 0.3995
D.D5: 0's = 0.9924, 1's = 0.0076
B.B11: 0's = 0.9959, 1's = 0.0041
D.D2: 0's = 0.9882, 1's = 0.0118
B.B4: 0's = 0.9271, 1's = 0.0729
D.D

In [25]:
## Rename all the response variables as target_X

# Total number of columns
total_cols = df_cleaned.shape[1]

# Generate new names for response variables
num_targets = total_cols - col_position
new_target_names = [f"target_{i+1}" for i in range(num_targets)]

# Apply the new names
df_cleaned.columns.values[col_position:] = new_target_names

In [26]:
df_cleaned.head(3)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,target_44,target_45,target_46,target_47,target_48,target_49,target_50,target_51,target_52,target_53
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df_cleaned.to_csv(path + "/Final_dataset.csv", index=False)